<a href="https://colab.research.google.com/github/ninjazone1/Giat-12-desa-kalirejo/blob/main/garis_pantai_demak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import geemap

In [2]:
ee.Authenticate()

In [3]:
ee.Initialize(project='of-30-483914')

In [4]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [6]:
wilayah = ee.Geometry.Point([110.525, -6.92])
Map.centerObject(wilayah, 13)

In [7]:
def apply_scale_factors(image):

    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)

    return image.addBands(optical_bands, None, True) \
                .addBands(thermal_bands, None, True)

In [8]:
def mask_clouds_l89(image):
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(1 << 1).eq(0) \
        .And(qa.bitwiseAnd(1 << 2).eq(0)) \
        .And(qa.bitwiseAnd(1 << 3).eq(0)) \
        .And(qa.bitwiseAnd(1 << 4).eq(0))
    return image.updateMask(mask)

def mask_clouds_l57(image):
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(1 << 1).eq(0) \
        .And(qa.bitwiseAnd(1 << 3).eq(0)) \
        .And(qa.bitwiseAnd(1 << 4).eq(0))
    return image.updateMask(mask)

In [16]:
def get_shoreline(year, satellite, color, layer_name):
    if satellite == 'L9':
        collection = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
        mask_func = mask_clouds_l89
        green, swir = 'SR_B3', 'SR_B6'
    elif satellite == 'L8':
        collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
        mask_func = mask_clouds_l89
        green, swir = 'SR_B3', 'SR_B6'
    elif satellite == 'L7':
        collection = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
        mask_func = mask_clouds_l57
        green, swir = 'SR_B2', 'SR_B5'
    else:
        collection = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
        mask_func = mask_clouds_l57
        green, swir = 'SR_B2', 'SR_B5'


    img = collection.filterBounds(roi) \
        .filterDate(f'{year}-04-01', f'{year}-10-30') \
        .map(apply_scale_factors) \
        .map(mask_func) \
        .median()


    mndwi = img.normalizedDifference([green, swir])
    water = mndwi.gt(0).selfMask()


    vectors = water.reduceToVectors(
        geometry=roi.buffer(10000),
        scale=30,
        maxPixels=1e9
    )


    outline = ee.Image().paint(vectors, 0, 2)
    Map.addLayer(outline, {'palette': color}, layer_name)

    return img

In [18]:
roi = wilayah
get_shoreline('1994', 'L5', 'FFFF00', '1994 (Awal)')
get_shoreline('2002', 'L7', 'FFCC00', '2002')
get_shoreline('2009', 'L5', 'FF9900', '2009')
get_shoreline('2017', 'L8', 'FF5500', '2017')
img_2024 = get_shoreline('2024', 'L9', 'FF0000', '2024 (Akhir)')